<a href="https://colab.research.google.com/github/StephMcCallum/MSE563-SM/blob/main/Week%203%20HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#lattice-mc
#square-lattice MC
#particles to sit on a grid
#calculate energy of your grid
#trial moves that obey detail balance
#have periodic boundary conditions

In [18]:
import numpy as np
import random
#object-oriented approach
class grid():
  #size and particles
  #helper function for neighbor sites to a particle
  def __init__(self,L,N):
    self.sites = np.zeros((L,L))
    self.N = N
    self.L = L
    self.particles = []
    for i in range(self.N):
      self.particles.append(particle(self))
      self.particles[i].location = [random.randint(0,L-1),random.randint(0,L-1)]
      self.sites[self.particles[i].location[0],self.particles[i].location[1]] = 1

  def pbc(self,s):
    if s>=self.L:
      s = s-self.L
    elif s < 0:
      s = s+self.L
    return s

  def microstates(self):
    count = 0
    for i in range(self.L):
      for j in range(self.L):
        if -1 < (i+1) < self.L:
          if self.sites[i,j] == 1 and self.sites[i+1,j] == 1:
            count += -1
        if -1 < (j+1) < self.L:
          if self.sites[i,j] == 1 and self.sites[i,j+1] == 1:
            count += -1
        if -1 < (i-1) < self.L:
          if self.sites[i,j] == 1 and self.sites[i-1,j] == 1:
            count += -1
        if -1 < (j-1) < self.L:
          if self.sites[i,j] == 1 and self.sites[i,j-1] == 1:
            count += -1
        if (i+1) >= self.L:
          #update coordinates with pbc
          coor = self.pbc(i+1)
          if self.sites[i,j] == 1 and self.sites[coor,j] == 1:
            count += -1
        if (j+1) >= self.L:
          coor = self.pbc(j+1)
          if self.sites[i,j] == 1 and self.sites[i,coor] == 1:
            count += -1
        if 0 > (i-1):
          coor = self.pbc(i-1)
          if self.sites[i,j] == 1 and self.sites[coor,j] == 1:
            count += -1
        if 0 > (j-1):
          coor = self.pbc(j-1)
          if self.sites[i,j] == 1 and self.sites[i,coor] == 1:
            count += -1
    E = count//2 #double counts pairs
    return E

In [32]:
class particle():
  def __init__(self,system):
    self.system = system
    self.location = [0,0]
    system.sites[self.location] == 1

  def available_neighbors(self,system):
    neighbors = []
    if system.sites[self.location[0],self.location[1]] == 1:
      i = self.location[0]
      j = self.location[1]
      neighbors.append([i+1,j])
      neighbors.append([i,j+1])
      neighbors.append([i-1,j])
      neighbors.append([i,j-1])
    for ind in neighbors:
      if system.L <= ind[0] or ind[0] < 0:
        neighbors[ind[0],ind[1]] == system.pbc(ind[0])
      if system.L <= ind[1] or ind[1] < 0:
        neighbors[ind[0],ind[1]] == system.pbc(ind[1])
     # if system.sites[ind[0],ind[1]] == 1:
       # neighbors.remove(ind)

    return neighbors

In [20]:
sample_system = grid(4,10)
print(sample_system.sites)
print(sample_system.microstates())

[[1. 1. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 1. 1.]
 [0. 1. 1. 0.]]
-10


In [30]:
import random
import scipy
import math
def sim(N,L,T,trials): #NVT Monte Carlo Function
  system = grid(L,N)

  N = N
  #trial move
  for i in range(trials):
    print(system.sites)
    random_particle = random.choice(range(N))
    print(random_particle,system.particles[random_particle].location)
    random_location = random.choice(system.particles[random_particle].available_neighbors(system))
    copy_system = system
    print(random_location)
    copy_system.sites[random_location[0],random_location[1]] = 1
    copy_system.sites[system.particles[random_particle].location] = 0
    copy_system.particles[random_particle].location == random_location
    #calculate energy
    delta_energy = copy_system.microstates() - system.microstates()
    boltzman_weight = - delta_energy / (scipy.constants.k * T)
    if np.random.random() < math.exp(boltzman_weight):
      #accept move
      print("accept")
      system == copy_system
      print(system.sites)
    else:
      continue

#detail balance would ensure move A->B has the same energy difference as B->A

In [34]:
sim(10,5,30,5)

[[0. 1. 0. 1. 1.]
 [1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
8 [2, 1]
[3, 1]
accept
[[0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
[[0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
8 [2, 1]


IndexError: Cannot choose from an empty sequence